In [1]:
import os
import torch
import numpy as np
from sklearn import preprocessing

In [2]:
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


## Loading Dataset

In [4]:
class Custom_Dataset(Dataset):
    def __init__(self, file, encode_categorical=False, train=False):
        self.train = train
        self.loadedFrom = file
        
        data = np.genfromtxt(file, delimiter=',', dtype='str')
        data = data[1:data.shape[0],]
        n_row, n_col = data.shape
        self.X, self.Y = data[:,0:n_col-1].astype(np.float64), data[:,n_col-1].astype(np.float64)
    
        # oe = preprocessing.OrdinalEncoder()
        # oe.fit(self.X)
        # self.X = oe.transform(self.X)
                
    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [5]:
train_path = './data/banknote/train.csv'
test_path = './data/banknote/test.csv'

In [6]:
dataset_train = Custom_Dataset(train_path, train=True)

In [7]:
train_batch_size = 1
train_size = len(dataset_train)

In [8]:
train_loader = DataLoader(dataset_train, batch_size=train_batch_size, shuffle=True)

# x_tr, y_tr = next(iter(train_loader))
# y_tr = torch.from_numpy(np.asarray(y_tr))
# print(y_tr)

In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_shape=14):
        super(NeuralNetwork, self).__init__()        
        self.stack = nn.Sequential(
            nn.Linear(input_shape, 10),
            nn.Sigmoid(),
            nn.Linear(10, 10),
            nn.Sigmoid(),
            nn.Linear(10, 1),
            nn.Sigmoid(),
        )
        
        for module in self.modules():
            if isinstance(module, nn.Linear):
                print(module)
                nn.init.uniform_(module.weight)
        
        self.loss = nn.L1Loss()
        self.double()
        
    def forward(self, x):        
        h = x
        for layer in self.stack:
            h = layer(h)
        return h

In [10]:
model = NeuralNetwork(input_shape=4).to(device)
print(model)

Linear(in_features=4, out_features=10, bias=True)
Linear(in_features=10, out_features=10, bias=True)
Linear(in_features=10, out_features=1, bias=True)
NeuralNetwork(
  (stack): Sequential(
    (0): Linear(in_features=4, out_features=10, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=10, out_features=1, bias=True)
    (5): Sigmoid()
  )
  (loss): L1Loss()
)


In [ ]:
learning_rate = 1

print("Starting Iteration")
for t in range(100):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    for i in range(train_size//train_batch_size):
        x_tr, y_tr = next(iter(train_loader))
        y_pred = model(x_tr).reshape(y_tr.shape)

        # Compute and print loss. We pass Tensors containing the predicted and true
        # values of y, and the loss function returns a Tensor containing the
        # loss.
        loss = model.loss(y_pred, y_tr)

        # Zero the gradients before running the backward pass.
        model.zero_grad()

        # Backward pass: compute gradient of the loss with respect to all the learnable
        # parameters of the model. Internally, the parameters of each Module are stored
        # in Tensors with requires_grad=True, so this call will compute gradients for
        # all learnable parameters in the model.
        loss.backward()

        # Update the weights using gradient descent. Each parameter is a Tensor, so
        # we can access its gradients like we did before.
        with torch.no_grad():
            learning_rate = learning_rate/1+t
            for param in model.parameters():
                param -= learning_rate * param.grad
    
    with torch.no_grad():
        random_ids = np.random.randint(train_size, size=train_batch_size)
        random_x, random_y = dataset_train[random_ids]
        pred = model(torch.from_numpy(random_x))
        print("pred", pred, "target", random_y)
        l = model.loss(pred, torch.from_numpy(random_y).to(torch.double))
        print("completing " + str(t) + " rounds, loss: " + str(pred))

Starting Iteration
pred tensor([[0.9997]], dtype=torch.float64) target [0.]
completing 0 rounds, loss: tensor([[0.9997]], dtype=torch.float64)


/Users/albertliu/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


pred tensor([[1.0000]], dtype=torch.float64) target [0.]
completing 1 rounds, loss: tensor([[1.0000]], dtype=torch.float64)
pred tensor([[1.0000]], dtype=torch.float64) target [1.]
completing 2 rounds, loss: tensor([[1.0000]], dtype=torch.float64)
pred tensor([[1.0000]], dtype=torch.float64) target [1.]
completing 3 rounds, loss: tensor([[1.0000]], dtype=torch.float64)
pred tensor([[1.0000]], dtype=torch.float64) target [0.]
completing 4 rounds, loss: tensor([[1.0000]], dtype=torch.float64)
pred tensor([[1.0000]], dtype=torch.float64) target [0.]
completing 5 rounds, loss: tensor([[1.0000]], dtype=torch.float64)
pred tensor([[1.0000]], dtype=torch.float64) target [1.]
completing 6 rounds, loss: tensor([[1.0000]], dtype=torch.float64)
pred tensor([[1.0000]], dtype=torch.float64) target [0.]
completing 7 rounds, loss: tensor([[1.0000]], dtype=torch.float64)
pred tensor([[1.0000]], dtype=torch.float64) target [1.]
completing 8 rounds, loss: tensor([[1.0000]], dtype=torch.float64)
pred ten

pred tensor([[0.]], dtype=torch.float64) target [0.]
completing 71 rounds, loss: tensor([[0.]], dtype=torch.float64)
pred tensor([[0.]], dtype=torch.float64) target [0.]
completing 72 rounds, loss: tensor([[0.]], dtype=torch.float64)
pred tensor([[0.]], dtype=torch.float64) target [1.]
completing 73 rounds, loss: tensor([[0.]], dtype=torch.float64)
pred tensor([[0.]], dtype=torch.float64) target [1.]
completing 74 rounds, loss: tensor([[0.]], dtype=torch.float64)
pred tensor([[0.]], dtype=torch.float64) target [1.]
completing 75 rounds, loss: tensor([[0.]], dtype=torch.float64)
pred tensor([[0.]], dtype=torch.float64) target [1.]
completing 76 rounds, loss: tensor([[0.]], dtype=torch.float64)
pred tensor([[0.]], dtype=torch.float64) target [1.]
completing 77 rounds, loss: tensor([[0.]], dtype=torch.float64)


In [ ]:
'''
class Test_Dataset(Dataset):
    def __init__(self, file, encode_categorical=False):
        self.loadedFrom = file

        data = np.genfromtxt(file, delimiter=',', dtype='str')
        data = data[1:data.shape[0],]
        n_row, n_col = data.shape
        self.id, self.X = data[:,0:1], data[:,1:n_col]
    
        oe = preprocessing.OrdinalEncoder()
        oe.fit(self.X)
        self.X = oe.transform(self.X)
                
    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx]
'''

class Test_Dataset(Dataset):
    def __init__(self, file, encode_categorical=False):
        self.loadedFrom = file

        data = np.genfromtxt(file, delimiter=',', dtype='str')
        data = data[1:data.shape[0],]
        n_row, n_col = data.shape
        self.X, self.Y = data[:,0:n_col-1].astype(np.float64), data[:,n_col-1].astype(np.float64)
    
        # oe = preprocessing.OrdinalEncoder()
        # oe.fit(self.X)
        # self.X = oe.transform(self.X)
                
    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [ ]:
dataset_test = Test_Dataset(test_path, encode_categorical=True)
test_loader = DataLoader(dataset_test, batch_size=1, shuffle=False)

In [ ]:
# Disable grad
with torch.no_grad():
    
    for i in range(100):
        _x, _y = np.asarray(dataset_test[i])
        item = torch.from_numpy(_x)

        # Generate prediction
        prediction = model(item)
        print("i=", prediction)

        # Predicted class value using argmax
        predicted_class = np.argmax(prediction)
        print("i=", predicted_class)